In [134]:
from mclass import *
from maze import *
from minHeapClass import *
import numpy as np
import random

In [2]:
def ManhattanDistance(a, b):
    return abs(a.x - b.x) + abs(a.y - b.y)

def neighbors(maze, current):
    children = []
    if not maze[current.left].isWall:
        children.append(maze[current.left])
    if not maze[current.right].isWall:
        children.append(maze[current.right])
    if not maze[current.down].isWall:
        children.append(maze[current.down])
    if not maze[current.up].isWall:
        children.append(maze[current.up])
    return children

def A_Star_Search(maze, startIdx, goalIdx):
    openList = MinHeap()
    gVal = {}
    parents = {}
    count = 0
    
    openList.push(maze[startIdx], 0)
    gVal[maze[startIdx]] = 0
    parents[maze[startIdx]] = None
    
    while not openList.isEmpty():
        current = openList.pop()
        count += 1
        
        if current.idx == goalIdx:
            break
            
        children = neighbors(maze, current)
        for child in children:
            newGVal = gVal[current] + 1
            if child not in gVal or newGVal < gVal[child]:
                gVal[child] = newGVal
                parents[child] = current
                fn = newGVal + ManhattanDistance(child, maze[goalIdx])
                openList.push(child, fn)
    
    return gVal, parents, count

def drawPath(maze, cellList, parents, startIdx, goalIdx):
    retPath = []
    current = cellList[goalIdx]

    while current != cellList[startIdx]:
        retPath.append(current)
        current = parents[current]
    
    newMaze = list(maze)

    for movement in retPath:
        if movement.idx == startIdx:
            continue
        if movement.idx == goalIdx:
            continue
        newMaze[movement.idx] = '.'

    maze_string = ''.join(newMaze)
    print maze_string
    
    return maze_string

In [30]:
print "Unsolved Maze"
maze, cellList, startIdx, goalIdx = getMaze('/inputMazes/openMaze.txt')
print maze + '\n'

print "Solved Maze"
gVals, parents, expanded = A_Star_Search(cellList, startIdx, goalIdx)
solution = drawPath(maze, cellList, parents, startIdx, goalIdx)
# gVals, parents, expanded = A_Star_Search(cellList, goalIdx, startIdx)
# solution = drawPath(maze, cellList, parents, goalIdx, startIdx)

print "Cost of Path:", gVals[cellList[goalIdx]]
print "Nodes Expanded:", expanded

f = open('a_star/a_star_open.txt', 'w')
f.write(solution)
f.write("\nCost of Path:" + " " + str(gVals[cellList[goalIdx]]))
f.write("\nNodes Expanded:" + " " + str(expanded))
f.close()

Unsolved Maze
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P            %
%                     %             %
%                     %             %
%                     %             %
%                     %             %
%                     %%%%%%%%      %
%                            %      %
%                            %      %
%                            %      %
%                            %      %
%                            %      %
%      %%%%%%%%%%%%%%%%%            %
%      %                            %
%      %                            %
%      %                            %
%      %                            %
%      %%%%                         %
%        .%                         %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Solved Maze
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
%                     %P            %
%                     %.            %
%                     %.            %
%                     %.            %
%                     %

In [19]:
# g = 1.1 search from start to each individual pellet
# h = sum of weights on MST from the remaining states. So say there is 3 pellets A,B,C. We calculate
# the MST from A, including B,C etc. And then the h value for each pellet is the sum of weights on the MST

In [150]:
maze2, cellList2, startIdx2, goalList = getMazeWithFinList('/inputMazes/tinySearch.txt')

In [151]:
def generateMST(lookup, root):
    retVal = []
    visited = []
    
    visited.append(root)
    
    while len(visited) != lookup.shape[0]:
        edges = []
        for vertex in visited:
            for adj in range(lookup.shape[0]):
                if adj not in visited and lookup[vertex][adj] != 0:
                    edges.append((vertex, adj))
        #add smallest edge
        smallestEdge = edges[0]
        for edge in edges:
            if lookup[edge[0]][edge[1]] < lookup[smallestEdge[0]][smallestEdge[1]]:
                smallestEdge = edge
            if lookup[edge[0]][edge[1]] == lookup[smallestEdge[0]][smallestEdge[1]]:
                x = random.randint(1,2)
                if x == 1:
                    continue
                else:
                    smallestEdge = edge
        retVal.append(smallestEdge)
        visited.append(smallestEdge[1])
    
    return retVal

In [162]:
gList = []
hList = []
fList = []
lookup = np.zeros((len(goalList), len(goalList)))

for finIdx in range(len(goalList)):
    gVals, parents, expanded = A_Star_Search(cellList2, startIdx2, goalList[finIdx])
    gList.append(gVals[cellList2[goalList[finIdx]]])

for x in range(len(goalList)):
    for y in range(len(goalList)):
        if x == y:
            lookup[x][y] = 0
            continue
        gVals, parents, expanded = A_Star_Search(cellList2, goalList[x], goalList[y])
        lookup[x][y] = gVals[cellList2[goalList[y]]]

for finIdx in range(len(goalList)):
    mstSum = 0
    edges = generateMST(lookup, finIdx)
    for edge in edges:
        mstSum += lookup[edge[0]][edge[1]]
    hList.append(mstSum)

for idx in range(len(gList)):
    fList.append(gList[idx] + hList[idx])
    
order = np.argsort(fList)

In [182]:
numbers = ['1','2','3','4','5','6','7','8','9']
upperCase = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ')
lowerCase = list('abcdefghijklmnopqrstuvwxyz')
ordering = numbers + upperCase + lowerCase

newMaze = list(maze2)

for idx in range(len(order)):
    newMaze[goalList[order[idx]]] = ordering[idx]

maze_string = ''.join(newMaze)
print maze_string

%%%%%%%%%%
%8  %   B%
% %2% %% %
% %   3%C%
% 4%P%   %
%5  1  6 %
% %%%% %9%
%A    7% %
%%%%%%%%%%
